<a href="https://colab.research.google.com/github/seamusrobertmurphy/VT0007-deforestation-map/blob/main/VT0007_deforestation_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment setup

In [13]:
!pip install leafmap geemap==0.16.4 geopandas numpy scikit-learn
#import geemap.tools as geemap_tools
import ee, json, geemap, ipyleaflet, os, numpy
from google.colab import drive
from google.colab import files
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
!drive.mount('/content/drive')

/bin/bash: -c: line 1: syntax error near unexpected token `'/content/drive''
/bin/bash: -c: line 1: `drive.mount('/content/drive')'


### Activate Earth Engine

In [9]:
!earthengine authenticate
#!ee.Authenticate() # now deprecated in certain Colab environments
ee.Initialize(project = "murphys-deforisk")

Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
Authenticate: Credentials already exist.  Use --force to refresh.


### Jurisidictional boundaries

In [10]:
country = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals("ADM0_NAME", "Guyana"))
states_all = country.aggregate_array('ADM1_NAME').distinct().getInfo()
states_all

['Barima Waini (region N°1)',
 'Cuyuni/mazaruni (region N°7)',
 'Demerara Mahaica (region N°4)',
 'East Berbice/corentyne (region N°6)',
 'Essequibo Islands/west Demerara (region N°3)',
 'Mahaica Berbice (region N°5)',
 'Pomeroon/supenaam (region N°2',
 'Potaro/siparuni (region N°8)',
 'Upper Demerara/berbice (region N°10)',
 'Upper Takutu/upper Essequibo (region N°9)']

In [34]:
state =  ee.FeatureCollection('FAO/GAUL/2015/level1').filter(
    ee.Filter.equals('ADM1_NAME', "Barima Waini (region N°1)"))
red = {"color": "red", "width": 2, "lineType": "solid", "fillColor": "00000000"}
white = {"color": "white", "width": 1, "lineType": "solid", "fillColor": "00000000"}
country_label = ee.FeatureCollection([ee.Feature(
    country.geometry().centroid(), {'country_name': country.first().get("ADM0_NAME").getInfo()})])

Map = geemap.Map()
Map.centerObject(country, 6)
Map.add_basemap('Esri.WorldImagery')
Map.addLayer(country.style(**white), {}, "Guyana")
Map.addLayer(state.style(**red), {}, "Barima Waini (region N°1)")
Map.add_labels(state,"ADM1_NAME",font_size="9pt",font_color="red",font_family="arial",font_weight="bold",)
Map.add_labels(country_label, "country_name", font_size="12pt", font_color="white", font_family="arial",)
Map

Map(center=[4.792974158655901, -58.97213873238909], controls=(WidgetControl(options=['position', 'transparent_…

The ImageCollection Landsat 8 Surface Reflectance Tier 1 have been atmospherically corrected using LaSRC and includes a cloud, shadow, water and snow mask produced using CFMASK, as well as a per-pixel saturation mask.

In [41]:
# derive masking, scaling, and ndvi function
def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI').toFloat()
    image = image.addBands(opticalBands, None, True) \
                 .addBands(thermalBands, None, True) \
                 .addBands(ndvi)
    return image.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI'],
        ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']
    ).updateMask(qaMask).updateMask(saturationMask)

# create collections for 2014 and 2024
collection_2014 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate('2014-01-01', '2014-12-31') \
                    .filterBounds(state) \
                    .map(maskL8sr)

collection_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                    .filterDate('2024-01-01', '2024-12-31') \
                    .filterBounds(state) \
                    .map(maskL8sr)

# median composites for 2014 and 2024
composite_2014 = collection_2014.select(['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']).median().clip(state).toFloat()
composite_2024 = collection_2024.select(['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']).median().clip(state).toFloat()


# visualization
ndviVis = {'min': 0.2, 'max': 0.8, 'palette': ['red', 'yellow', 'green']}
Map = geemap.Map()
Map.centerObject(state, 8)
Map.addLayer(composite_2014.select('NDVI'), ndviVis, 'NDVI 2014')
Map.addLayer(composite_2024.select('NDVI'), ndviVis, 'NDVI 2024')
Map.addLayer(state, {}, 'Area of Interest')
Map.addLayerControl()
Map

Map(center=[7.63219842740881, -59.75025883718175], controls=(WidgetControl(options=['position', 'transparent_b…

In [65]:
# confirm dates, scene IDs, band names of images
firstImage_2014 = collection_2014.first()
sceneId_2014 = firstImage_2014.get('system:index').getInfo()
print(f"Scene ID for collection_2014: {sceneId_2014}")

firstImage_2024 = collection_2024.first()
sceneId_2024 = firstImage_2024.get('system:index').getInfo()
print(f"Scene ID for collection_2024: {sceneId_2024}")

bandNames_2014 = composite_2014.bandNames().getInfo()
print(f"Band names: {bandNames_2014}")

bandNames_2024 = composite_2024.bandNames().getInfo()
print(f"Band names: {bandNames_2024}")

Scene ID for collection_2014: LC08_231055_20140111
Scene ID for collection_2024: LC08_231055_20240107
Band names: ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']
Band names: ['BLUE', 'GREEN', 'RED', 'NIR08', 'SWIR16', 'SWIR22', 'NDVI']


In [70]:
from datetime import datetime

# extract pathrow and date from scene ID
def get_pathrow_date(image_collection):
  first_image = image_collection.first()
  scene_id = first_image.get('system:index').getInfo()
  parts = scene_id.split('_')
  pathrow = parts[1]
  date_str = parts[2]
  date_obj = datetime.strptime(date_str, '%Y%m%d')
  date = date_obj.strftime('%Y-%m-%d')
  return pathrow, date

# define export parameters
def define_export_params(image, pathrow, date, band_name):
  return {
    'image': image.select(band_name),
    'description': f'composite_{date}_{band_name}_export',
    'bucket': 'deforisk_bucket_1',
    'fileNamePrefix': f'LANDSAT_TM-ETM-OLI_{pathrow}_{band_name}_{date}',
    'scale': 30,
    'region': state.geometry(),
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
  }

# get pathrow and date for each collection
pathrow_2014, date_2014 = get_pathrow_date(collection_2014)
pathrow_2024, date_2024 = get_pathrow_date(collection_2024)

# get band names
bandNames_2014 = composite_2014.bandNames().getInfo()
bandNames_2024 = composite_2024.bandNames().getInfo()

# export to cloud bucket
for band_name in bandNames_2014:
    export_params_2014 = define_export_params(composite_2014, pathrow_2014, date_2014, band_name)
    task_2014 = ee.batch.Export.image.toCloudStorage(**export_params_2014)
    task_2014.start()
    print(f"Exporting 2014 image for band {band_name}. Task ID: {task_2014.id}")

for band_name in bandNames_2024:
    export_params_2024 = define_export_params(composite_2024, pathrow_2024, date_2024, band_name)
    task_2024 = ee.batch.Export.image.toCloudStorage(**export_params_2024)
    task_2024.start()
    print(f"Exporting 2024 image for band {band_name}. Task ID: {task_2024.id}")

# export full stack to drive
#export_params_drive = {
#    'image': composite_2014,
#    'description': 'export_2014',
#    'folder': 'VT0007-deforestation-map',
#    'fileNamePrefix': 'LANDSAT-C2-L2_OLI_TILE_BAND_2014-11-11',
#    'scale': 30,
#    'region': state.geometry(),
#    'maxPixels': 1e13,
#    'fileFormat': 'GeoTIFF',
#    'formatOptions': {'cloudOptimized': True}
#}

#task = ee.batch.Export.image.toDrive(**export_params_drive)
#task.start()
#print(f"Exporting image. Task ID: {task.id}")

Exporting 2014 image for band BLUE. Task ID: 6EA3PHOOJSSPSK62ZG73RM6G
Exporting 2014 image for band GREEN. Task ID: GCFMAH3UGO44UC55YLVZLW6C
Exporting 2014 image for band RED. Task ID: UPO5UJ4T3CLW6C6BFKISD3DF
Exporting 2014 image for band NIR08. Task ID: VSW7NGIQCALO7OD72OVMRVYQ
Exporting 2014 image for band SWIR16. Task ID: C5QAPB24TKTAS7EG6MIHC3GF
Exporting 2014 image for band SWIR22. Task ID: SUDF3XE45KNXJYIHUV4SCROO
Exporting 2014 image for band NDVI. Task ID: R2YUC2SMNLERTO2BPLCS74WE
Exporting 2024 image for band BLUE. Task ID: VQ5B74MLOJ6ZKLN3BOTIXAAC
Exporting 2024 image for band GREEN. Task ID: Y4JZGRXOAEAIIXOVR5I7ID3W
Exporting 2024 image for band RED. Task ID: B676JUQXPFLDD2ICYNLD54IC
Exporting 2024 image for band NIR08. Task ID: JJVTBGXY7TAHQL3IUU4ZFWCC
Exporting 2024 image for band SWIR16. Task ID: 574U4KUEBBERV6DJDFITLTYW
Exporting 2024 image for band SWIR22. Task ID: C7HFJWINW7OXEYYTOPTET3LB
Exporting 2024 image for band NDVI. Task ID: 3JK3BNYWQURBZ4GCUXG4YBXE
